# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
## SEMILLERIO - 64 árboles.

In [2]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-01 21:39:27 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,659104,35.2,1454570,77.7,1454570,77.7
Vcells,1224118,9.4,8388608,64.0,1975163,15.1


In [3]:
plocal <- list()

# 501
plocal$qcanaritos <- 5L
plocal$min_data_in_leaf <- 200L
plocal$learning_rate <- 1.0
plocal$gradient_bound <- 0.4


plocal$APO <- 1
plocal$ksemillerio <- 50

In [4]:
PARAM <- list()
PARAM$experimento <- "apo-505_C3_n7"
PARAM$semilla_primigenia <- 888887

In [5]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [6]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset1 <- fread("~/datasets/competencia_02_crudo.csv.gz" )
dataset2<- fread("~/datasets/competencia_03_crudo.csv.gz" )

dataset <- rbind(dataset1, dataset2)


# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-12-01 21:39:28 UTC"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,768268,41.1,1454570,77.7,1454570,77.7
Vcells,1301334262,9928.4,1777924524,13564.5,1449740053,11060.7


[1] "2025-12-01 21:39:58 UTC"

In [7]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

In [8]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [9]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-12-01 21:40:00 UTC"

In [10]:
cols <- names(dataset)

# encontrar columnas que comienzan con "c"
c_vars <- grep("^c", cols, value = TRUE)

# para cada columna cXXX, verificar si existe mXXX
pairs <- lapply(c_vars, function(cname){
  mname <- sub("^c", "m", cname)  # reemplaza el prefijo c → m
  if (mname %in% cols) {
    list(c = cname, m = mname)
  } else {
    NULL
  }
})


pairs <- Filter(Negate(is.null), pairs)

# crear las columnas ratio
for(p in pairs){
  cname <- p$c
  mname <- p$m
  rname <- paste0("ratio_", sub("^c", "", cname)) 
  dataset[, (rname) := get(mname) / pmax(get(cname), 1)]
}

ncol(dataset)

[1] 178

In [11]:
head(dataset)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,ratio_comisiones_otras,ratio_forex_buy,ratio_forex_sell,ratio_transferencias_recibidas,ratio_transferencias_emitidas,ratio_extraccion_autoservicio,ratio_cheques_depositados,ratio_cheques_emitidos,ratio_cheques_depositados_rechazados,ratio_cheques_emitidos_rechazados
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
250149174,201901,1,0,0,55,296,464.44,31108.76,1497.55,⋯,374.3875,0,0,0,0.0,0,3306.19,0,3306.19,0
252045720,201901,1,0,0,75,251,888.71,9787.31,848.09,⋯,141.3483,0,0,0,0.0,0,0.00,0,0.00,0
254876531,201901,1,0,1,49,53,1964.85,18571.31,1211.57,⋯,151.4462,0,0,0,0.0,0,0.00,0,0.00,0
256785538,201901,1,0,0,67,117,663.04,43899.63,649.36,⋯,324.6800,0,0,0,0.0,0,0.00,0,0.00,0
258845364,201901,1,0,1,65,258,732.51,36029.59,116.13,⋯,19.3550,0,0,0,11143.5,0,0.00,0,0.00,0
263194659,201901,1,0,1,48,274,1124.66,11287.21,536.56,⋯,53.6560,0,0,0,0.0,3128,0.00,0,0.00,0


In [12]:
select_col_montos <- function(dt) {
  
  todas_cols <- names(dt)
  
 # Patrón regex combinado
  patron <- "^m|Master_m|Visa_m|ratio_" 
  cols_montos <- grep(patron, todas_cols, value = TRUE)
  
  # Excluir
  excluir <- c("numero_de_cliente", "foto_mes")
  cols_montos <- setdiff(cols_montos, excluir)
  
  message(sprintf("Columnas de montos seleccionadas: %d", length(cols_montos)))
  
  return(cols_montos)
}

In [13]:
library(data.table)

feature_engineering_rank_pos <- function(dt, columnas, columna_mes = "foto_mes", replace = FALSE) {

  for (col in columnas) {
    
    new_col <- if (replace) col else paste0("pct_rank_", col)

    dt[, (new_col) :=
         dt[, {
           x  <- get(col)
           sg <- sign(x)

           # resultado por cada signo dentro del mes
           r <- numeric(.N)

           # negativos
           neg <- sg == -1 & !is.na(x)
           if (sum(neg) > 1) {
             rk <- frank(x[neg], ties.method = "min")
             r[neg] <- (rk - 1) / (sum(neg) - 1)
           } else if (sum(neg) == 1) {
             r[neg] <- 0
           }

           # positivos
           pos <- sg == 1 & !is.na(x)
           if (sum(pos) > 1) {
             rk <- frank(x[pos], ties.method = "min")
             r[pos] <- (rk - 1) / (sum(pos) - 1)
           } else if (sum(pos) == 1) {
             r[pos] <- 0
           }

           # ceros → 0
           r[x == 0] <- 0

           # NA → NA
           r[is.na(x)] <- NA_real_

           r
         }, 
         by = get(columna_mes)][, V1] ]
  }

  dt
}


In [14]:
columnas_monto = select_col_montos(dataset)
dataset = feature_engineering_rank_pos(dataset, columnas_monto, replace = TRUE)

Columnas de montos seleccionadas: 95



In [15]:
head(dataset)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,ratio_comisiones_otras,ratio_forex_buy,ratio_forex_sell,ratio_transferencias_recibidas,ratio_transferencias_emitidas,ratio_extraccion_autoservicio,ratio_cheques_depositados,ratio_cheques_emitidos,ratio_cheques_depositados_rechazados,ratio_cheques_emitidos_rechazados
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
250149174,201901,1,0,0,55,296,0.08733824,0.7850076,0.79531410,⋯,0.9310676,0,0,0,0.0000000,0.000000,0.08015708,0,0.03019538,0
252045720,201901,1,0,0,75,251,0.20294201,0.2675373,0.56797326,⋯,0.7450561,0,0,0,0.0000000,0.000000,0.00000000,0,0.00000000,0
254876531,201901,1,0,1,49,53,0.51463528,0.5707132,0.72179286,⋯,0.7675762,0,0,0,0.0000000,0.000000,0.00000000,0,0.00000000,0
256785538,201901,1,0,0,67,117,0.13309633,0.8725093,0.44505060,⋯,0.9142133,0,0,0,0.0000000,0.000000,0.00000000,0,0.00000000,0
258845364,201901,1,0,1,65,258,0.15443461,0.8273827,0.09684969,⋯,0.1222481,0,0,0,0.8004331,0.000000,0.00000000,0,0.00000000,0
263194659,201901,1,0,1,48,274,0.28169178,0.3274923,0.39247529,⋯,0.3902831,0,0,0,0.0000000,0.555298,0.00000000,0,0.00000000,0


### Feature Engineering Historico

In [16]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [17]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [18]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [19]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-12-01 21:44:43 UTC"

In [20]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 3
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- TRUE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [21]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1178

[1] "2025-12-01 21:46:07 UTC"

## Modelado

No hay modelado, no se hace optimizacion de hiperparametros.

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202104]
* Se realiza undersampling al 5%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

### Mid Training Strategy

In [22]:
PARAM$train_final$future <- c(202107)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, #202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202105
)

PARAM$train_final$undersampling <- 0.2  # Silvana y Sofia

In [23]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

In [24]:
# canaritos
PARAM$train_final$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
    set.seed(PARAM$semilla_primigenia)
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
  setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-12-01 21:46:15 UTC"

#### Registros cambio las proporciones de POS/NEG

In [25]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [26]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [27]:
PARAM$train_final$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,
    zero_as_missing = TRUE,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 64L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_final$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)

Sys.time()

[1] "2025-12-01 21:46:15 UTC"

In [28]:
# Semillerio Final
PARAM$train_final$APO <- plocal$APO
PARAM$train_final$ksemillerio  <- plocal$ksemillerio

PARAM$train_final$cortes <- c(8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000)

In [29]:
# detach("package:lightgbm", unload= TRUE)

In [30]:
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")

Loading required package: zlightgbm



In [31]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [32]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$APO*PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 613507 487481 427213 435593 428429 556727 272329 359419 951001 471439
[11] 214063 819061 697441 805991 111427 548897 671471 371857 614909 969377
[21] 575401 272567 240997 889579 900349 582821 974599 775063 483727 815953
[31] 649087 107137 874729 559259 628787 770981 722663 568913 420073 762253
[41] 868529 155377 792223 845683 610823 938881 669611 559523 119849 327443

In [33]:
campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [34]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 808190 columnas 1182 


[1] "2025-12-01 21:46:30 UTC"

In [35]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$lgbm$param_completo)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_mid", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-12-01 23:17:43 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [36]:
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_final$lgbm$qcanaritos > 0 ) {
    set.seed(PARAM$semilla_primigenia)
  for( i in seq(PARAM$train_final$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [37]:
# dataset de future, donde en este caso estoy haciendo testing

mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [38]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion_mid.txt") )
  file.remove("prediccion_mid.txt")

In [39]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_mid", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion_mid.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-12-01 23:20:01 UTC"

In [40]:
mganancias

401600000,405200000,401600000,404400000,405600000,4.1e+08,4.08e+08,406800000,410400000


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [41]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion_mid.txt")

In [42]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 11000

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_mid_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-12-01 23:20:02 UTC"

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [43]:
colmedias <- colMeans( mganancias, na.rm=TRUE )
mcorte_mejor <- max(colmedias, na.rm=TRUE)
icorte_mejor <- which.max( colmedias )
corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [44]:
tbl <- as.data.table( as.list( colmedias ) )
colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
tbl[, experimento := PARAM$experimento ]

exp_gral <- "/content/buckets/b1/exp/apo-gral"
dir.create(exp_gral, showWarnings=FALSE)
fwrite( tbl,
  file= paste0( exp_gral, "/tb_experimentos.txt"),
  sep= "\t",
  append= TRUE
)

In [45]:
colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
tbl_local <- as.data.table( mganancias )

fwrite( tbl_local,
  file= "tb_apo.txt",
  sep= "\t"
)

In [46]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

mcorte_mejor
icerca
tb_prediccion[ icerca]

[1] 410400000

[1] 10240

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
657806794,202107,1,0.1218158,410400000


In [47]:
icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
icerca

[1] 10240

In [48]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:icerca, Predicted := 1L] # marco los primeros

archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_mid_",  icerca, ".csv")

# grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_pseudo_kaggle,
  sep= ","
)


In [49]:
# la subida a Kaggle
#comando <- "kaggle competitions submit"
#competencia <- "-c  test-202106"
#arch <- paste( "-f", archivo_pseudo_kaggle)
#mensaje <-  paste0( "-m 'exp=", PARAM$experimento,
#  "  ", paste(names(plocal), plocal, sep= "=", collapse= ";" ),
#  " envios=", icorte_mejor,"'")

                    
#linea <- paste( comando, competencia, arch, mensaje)
#salida <- system(linea, intern=TRUE)
#cat(salida)

In [50]:
Sys.time()

[1] "2025-12-01 23:20:03 UTC"

In [51]:
PARAM$train_prod$future <- c(202109)

PARAM$train_prod$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, #202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202105, 202106, 202107
)

PARAM$train_prod$undersampling <- 0.2  # Silvana y Sofia

In [52]:
# se filtran los meses donde se entrena el modelo productivo
dataset_train_prod <- dataset[foto_mes %in% PARAM$train_prod$training]

In [53]:
# canaritos
PARAM$train_prod$lgbm$qcanaritos <- plocal$qcanaritos

cols0 <- copy(colnames(dataset_train_prod))
filas <- nrow(dataset_train_prod)

if( PARAM$train_prod$lgbm$qcanaritos > 0 ) {
    set.seed(PARAM$semilla_primigenia)
  for( i in seq(PARAM$train_prod$lgbm$qcanaritos) ){
    dataset_train_prod[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dataset_train_prod), cols0 ) )
  setcolorder( dataset_train_prod, c( cols_canaritos, cols0 ) )
}

Sys.time()

[1] "2025-12-01 23:20:12 UTC"

In [54]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_prod[, azar := runif(nrow(dataset_train_prod))]
dataset_train_prod[, training := 0L]

dataset_train_prod[
  (azar <= PARAM$train_prod$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_prod[, azar:= NULL] # elimino la columna azar

In [55]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_prod[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [56]:
PARAM$train_prod$lgbm$param_completo <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,
    zero_as_missing = TRUE,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= plocal$min_data_in_leaf,  #este ya es el valor default de LightGBM

  num_iterations= 64L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= plocal$learning_rate,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$train_prod$lgbm$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= plocal$gradient_bound   # default de zLightGBM
)


# Semillerio Productivo
PARAM$train_prod$APO <- plocal$APO
PARAM$train_prod$ksemillerio  <- plocal$ksemillerio

Sys.time()

[1] "2025-12-01 23:20:13 UTC"

In [57]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_prod$semillas <- sample(primos)[seq( PARAM$train_prod$APO*PARAM$train_prod$ksemillerio )]
PARAM$train_prod$semillas

[1] 613507 487481 427213 435593 428429 556727 272329 359419 951001 471439
[11] 214063 819061 697441 805991 111427 548897 671471 371857 614909 969377
[21] 575401 272567 240997 889579 900349 582821 974599 775063 483727 815953
[31] 649087 107137 874729 559259 628787 770981 722663 568913 420073 762253
[41] 868529 155377 792223 845683 610823 938881 669611 559523 119849 327443

In [58]:
campos_buenos <- setdiff(
  colnames(dataset_train_prod),
  c( "clase_ternaria", "clase01", "training", "azar")
)

In [59]:
# dejo los datos en formato LightGBM
dtrain_prod <- lgb.Dataset(
  data= data.matrix(dataset_train_prod[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_prod[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_prod), "columnas", ncol(dtrain_prod), "\n")
Sys.time()

filas 877618 columnas 1182 


[1] "2025-12-01 23:20:50 UTC"

In [60]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_prod$lgbm$param_completo)

for( sem in PARAM$train_prod$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_prod", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_prod,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

In [ ]:
#Canaritos al dataset de predicción

dfuture <- dataset[foto_mes %in% PARAM$train_prod$future ]

cols0 <- copy(colnames(dfuture))
filas <- nrow(dfuture)

if( PARAM$train_prod$lgbm$qcanaritos > 0 ) {
    set.seed(PARAM$semilla_primigenia)
  for( i in seq(PARAM$train_prod$lgbm$qcanaritos) ){
    dfuture[, paste0("canarito_",i) := runif( filas) ]
  }

  # las columnas canaritos mandatoriamente van al comienzo del dataset
  cols_canaritos <- copy( setdiff( colnames(dfuture), cols0 ) )
  setcolorder( dfuture, c( cols_canaritos, cols0 ) )
}

In [ ]:
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [ ]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_prod$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_prod$ksemillerio
  hasta <- desde + PARAM$train_prod$ksemillerio - 1
  semillas <- PARAM$train_prod$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_prod", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_prod <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_prod, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_prod)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [ ]:
# genero archivos de predicción
envios <- 11000

for( vapo in seq(PARAM$train_prod$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv") 

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

In [ ]:
head(dataset)

In [ ]:
dt_val <- dataset[foto_mes == 202107, .(numero_de_cliente, clase_ternaria, foto_mes)]

archivo_val <- fread("~/buckets/b1/exp/apo-505_C3_n7/kaggle/KAapo-505_C3_n7_mid_1_11000.csv")

In [ ]:
dt_merged <- merge(dt_val, archivo_val, 
                   by = "numero_de_cliente",
                   all.x = TRUE)

In [ ]:
dt_merged[, clase_binaria :=
             fifelse(clase_ternaria == "BAJA+2", 1, 0)]

In [ ]:
conf_matrix <- table(
  Real = dt_merged$clase_binaria,
  Predicted = dt_merged$Predicted
)

conf_matrix

In [ ]:
accuracy <- sum(dt_merged$clase_binaria == dt_merged$Predicted) / nrow(dt_merged)
accuracy

In [ ]:
TP <- sum(dt_merged$clase_binaria == 1 & dt_merged$Predicted == 1)
FP <- sum(dt_merged$clase_binaria == 0 & dt_merged$Predicted == 1)
FN <- sum(dt_merged$clase_binaria == 1 & dt_merged$Predicted == 0)
TN <- sum(dt_merged$clase_binaria == 0 & dt_merged$Predicted == 0)

precision <- TP / (TP + FP)
recall <- TP / (TP + FN)
F1 <- 2 * precision * recall / (precision + recall)

precision; recall; F1